##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TFRecord dan tf.Example

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/load_data/tfrecord"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />Lihat di TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/load_data/tfrecord.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Jalankan di Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/load_data/tfrecord.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />Lihat source di GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/load_data/tfrecord.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Unduh notebook</a>
  </td>
</table>

Membaca data secara efisien akan membantu untuk menjadikan data Anda terserialisasi dan menyimpannya dalam kumpulan file (100 hingga 200MB) yang dapat dibaca secara linear. Hal ini dapat dilakukan apabila data di kirimkan melalui jaringan. Hal ini juga dapat berguna untuk proses penyimpanan sementara setiap *data-preprocessing*.

Format TFRecord adalah format sederhana untuk menyimpan urutan dari record biner.

[Protocol buffers](https://developers.google.com/protocol-buffers/) adalah sebuah *library* antarplatform dan antarbahasa untuk menserialisasi data yang terstruktur secara efisien.

Pesan protokol didefinisikan sebagai file `.proto`, ini sering menjadi cara termudah untuk memahami sebuah tipe pesan.

Pesan `tf.Example` (atau protobuf) adalah tipe pesan yang fleksibel yang merepresentasikan pemetaan `{"string": value}`. Hal ini didesain untuk digunakan dengan Tensorflow dan digunakan diseluruh level API yang lebih tinggi seperti [TFX](https://www.tensorflow.org/tfx/).


This notebook will demonstrate how to create, parse, and use the `tf.Example` message, and then serialize, write, and read `tf.Example` messages to and from `.tfrecord` files.
Notebook ini akan mendemonstrasikan bagaimana cara membuat, menguraikan, dan menggunakan pesan `tf.Example`, dan kemudian menserilaisasi, menulis, dan membaca pesan `tf.Example` kepada dan dari file `.tfrecord`.

Note: While useful, these structures are optional. There is no need to convert existing code to use TFRecords, unless you are using [`tf.data`](https://www.tensorflow.org/guide/datasets) and reading data is still the bottleneck to training. See [Data Input Pipeline Performance](https://www.tensorflow.org/guide/performance/datasets) for dataset performance tips.
Catatan: Meskipun bermanfaat, struktur ini opsional. Tidak perlu untuk mengubah kode yang ada untuk menggunakan TFRecords, kecuali anda menggunakan [`tf.data`](https://www.tensorflow.org/guide/datasets) dan membaca data masih menyebabkan kemacetan pada saat training. Lihat [Data Input Pipeline Performance](https://www.tensorflow.org/guide/performance/datasets) untuk kiat kinerja dataset.

## Pengaturan

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version hanya ada di Colab
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf

import numpy as np
import IPython.display as display

## `tf.Example`

### Tipe data untuk `tf.Example`

Pada dasarnya, `tf.Example` adalah pemetaan `{"string": tf.train.Feature}`

Tipe pesan `tf.train.Feature` dapat menerima salah satu dari tiga tipe berikut (Sebagai referensi lihat [`.proto` file](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/core/example/feature.proto)). Sebagian besar tipe ini dapat diubah menjadi salah satu dari berikut:


1. `tf.train.BytesList` (tipe berikut dapat diubah)

  - `string`
  - `byte`

1. `tf.train.FloatList` (tipe berikut dapat diubah)

  - `float` (`float32`)
  - `double` (`float64`)

1. `tf.train.Int64List` (tipe berikut dapat diubah)

  - `bool`
  - `enum`
  - `int32`
  - `uint32`
  - `int64`
  - `uint64`

Untuk mengkonversi tipe standar Tensorflow menjadi `tf.Example`- `tf.train.Feature` kompatibel, Anda dapat menggunakan fungsi pintas dibawah ini. Perhatikan bahwa setiap fungsi mengambil inputan berupa nilai skalar dan mengembalikan `tf.train.Feature` yang berisi salah satu dari tiga tipe `list` diatas:

In [ ]:
# Fungsi Berikut dapat digunakan untuk mengkonversi sebuah nilau menjadi tipe yang kompatibel
# dengan tf.Example.
def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList tidak membuka string dari EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

Catatan: Agar tetap sederhana, contoh ini hanya menggunakan input skalar. Cara yang paling mudah untuk menangani fitur yang bukan skalar adalah dengan menggunakan `tf.serialize_tensor` untuk mengkonversi tensors menjadi string biner. String adalah skalar di dalam tensorflow. Gunakan `tf.parse_tensor` untuk mengkonversi string biner kembali ke tensor.

Dibawah ini beberapa contoh bagaimana fungsi-fungsi tersebut bekerja. Perhatikan berbagai jenis tipe input dan dan tipe output terstandardisasi. Jika tipe input untuk fungsi tidak cocok dengan salah satu tipe yang dapat diubah di atas, fungsi akan menampilkan eksepsi (contoh: `_int64_feature(1.0)` akan menghsailkan error, karena `1.0` bertipe float, sehingga seharusnya digunakan fungsi `_float_feature`):

In [ ]:
print(_bytes_feature(b'test_string'))
print(_bytes_feature(u'test_bytes'.encode('utf-8')))

print(_float_feature(np.exp(1)))

print(_int64_feature(True))
print(_int64_feature(1))

Semua pesan proto dapat diserialisasi menjadi string biner menggunakan *method* `.SerializeToString`:

In [ ]:
feature = _float_feature(np.exp(1))

feature.SerializeToString()

### Membuat pesan `tf.Example` 

Misalkan anda ingin membuat pesan `tf.Example` dari data yang ada. Dalam praktiknya, dataset dapat datang dari mana saja, tetapi prosedur dalam pembuatan pesan `tf.Example` dari sebuah pengamatan akan sama:

1. Dalam setiap pengamatan, setiap nilai harus dikonversi menjadi sebuah `tf.train.Feature` berisi salah satu 3 tipe kompatibel, menggunakan satu dari fungsi-fungsi diatas.

1. Anda membuat peta (*dictionary*) dari string nama fitur menjadi nilai fitur yang dikodekan yang diproduksi pada #1.

1. Peta yang diproduksi pada langakah 2 dikonversi menjadi [`Features` message](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/core/example/feature.proto#L85).

Dalam notebook ini, Anda akan membuat dataset menggunakan NumPy

Dataset ini memilik 4 fitur:

* fitur boolean, `False` atau `True` dengan probabilitas yang sama
* fitur integer secara berdistrbusi uniform dan acak dipilih dari nilai `[0, 5]`
* fitur string dibuat dari tabel string menggunakan fitur integer sebagai index
* fitur float dari berdistribusi normal standar

Pertimbangkan sampel berisi 10,000 pengamatan independen dan terdistribusi secara identik dari setiap distribusi di atas:

In [ ]:
# Jumlah observasi di dalam dataset
n_observations = int(1e4)

# Fitur Boolean, dikodekan sebagai False atau True.
feature0 = np.random.choice([False, True], n_observations)

# Fitur integer, acak dari 0 sampai 4.
feature1 = np.random.randint(0, 5, n_observations)

# Fitur string
strings = np.array([b'cat', b'dog', b'chicken', b'horse', b'goat'])
feature2 = strings[feature1]

# Fitur float, dari distribusi normal standar
feature3 = np.random.randn(n_observations)

Setiap fitur ini dapat diubah menjadi sebuah tipe kompatibel `tf.Example`- menggunakan salah satu dari `_bytes_feature`, `_float_feature`, `_int64_feature`. Anda kemudian dapat membuat pesan `tf.Example` dari fitur yang dikodekan ini:

In [ ]:
def serialize_example(feature0, feature1, feature2, feature3):
  """
  Creates a tf.Example message ready to be written to a file.
  """
  # Membuat *dictionary* yang memetakan fitur nama ke tipe data kompatible tf.Example
  feature = {
      'feature0': _int64_feature(feature0),
      'feature1': _int64_feature(feature1),
      'feature2': _bytes_feature(feature2),
      'feature3': _float_feature(feature3),
  }

  # Membuat pesan Fitur menggunakan tf.train.Example.

  example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
  return example_proto.SerializeToString()

Sebagai contoh, misalkan anda mempunyai pengamatan dari dataset, `[False, 4, bytes('goat'), 0.9876]`. Anda dapat membuat dan menampilkan pesan `tf.Example` untuk pengmatan ini menggunakan `create_message()`. Setiap pengamatan akan ditulis sebagai pesan `Features` seperti diatas. Perhatikan bahwa `tf.Example` [message](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/core/example/example.proto#L88) adalah sebuah pembungkus di sekeliling pesan `Features`:

In [ ]:
# Ini adalah contoh pengamatan dari dataset.

example_observation = []

serialized_example = serialize_example(False, 4, b'goat', 0.9876)
serialized_example

Untuk melakukan dekode gunakan *method* `tf.train.Example.FromString`

In [ ]:
example_proto = tf.train.Example.FromString(serialized_example)
example_proto

## Detail Format TFRecords

file TFRecord berisi sebuah urutan dari record. File hanya dapat dibaca secara berurutan.

Setiap record berisi byte-string, untuk data-payload, ditambah data-length, dan hash CRC32C (32-bit CRC menggunakan Castagnoli polynomial) untuk mengecek integritas.

Setiap record disimpan dalam format berikut:

    uint64 length
    uint32 masked_crc32_of_length
    byte   data[length]
    uint32 masked_crc32_of_data

Record digabungkan untuk memproduksi file.  CRCs 
[dijelaskan disini](https://en.wikipedia.org/wiki/Cyclic_redundancy_check), dan *mask* dari CRC adalah:

    masked_crc = ((crc >> 15) | (crc << 17)) + 0xa282ead8ul

Catatan: Tidak ada syarat untuk menggunakan `tf.Example` pada file TFRecord. `tf.Example` adalah *method* untuk proses serialisasi *dictionary* menjadi byte-string. Baris teks, data enkode gambar, atau tensor terserialisasi (menggunakan `tf.io.serialize_tensor`, dan `tf.io.parse_tensor` when loading). See the `tf.io` module for more options. `tf.io.parse_tensor` ketika memuat). Lihat modul `tf.io` untuk opsi lebih.

## File TFRecord menggunakan `tf.data`

Modul `tf.data` juga menyediakan alat untuk membaca dan menulis data di TensorFlow.

### Menulis file TFRecord

Cara termudah untuk mendapatkan data menjadi dataset adalah menggunakan *method* `from_tensor_slices`.

Digunakan pada array, *method* ini mengembalikan dataset skalar:

In [ ]:
tf.data.Dataset.from_tensor_slices(feature1)

Digunakan pada sebuah array tuple, mengembalikan dataset tuple:

In [ ]:
features_dataset = tf.data.Dataset.from_tensor_slices((feature0, feature1, feature2, feature3))
features_dataset

In [ ]:
# Gunakan `take(1)` untuk mengambil satu contoh dari dataset.
for f0,f1,f2,f3 in features_dataset.take(1):
  print(f0)
  print(f1)
  print(f2)
  print(f3)

Gunakan *method* `tf.data.Dataset.map` untuk menggunakan fungsi ke setiap elemen sebuah `Dataset`.

Fungsi yang terpetakan harus beroprasi dalam mode grafik TensorFlow —fungsi beroperasi dan mengembalikan `tf.Tensors`. Fungsi non-tensor, seperti `serialize_example`, dapat dibungkus dengan `tf.py_function` agar menjadi kompatibel.

Menggunakan `tf.py_function` memerlukan penentuan bentuk dan tipe informasi yang tidak tersedia:

In [ ]:
def tf_serialize_example(f0,f1,f2,f3):
  tf_string = tf.py_function(
    serialize_example,
    (f0,f1,f2,f3),  # berikan args ini ke fungsi yang di atas.
    tf.string)      # tipe yang dikembalikan adalah `tf.string`.
  return tf.reshape(tf_string, ()) # Hasilnya adalah skalar

In [ ]:
tf_serialize_example(f0,f1,f2,f3)

Gunakan fungsi ini ke setiap elemen di dataset:

In [ ]:
serialized_features_dataset = features_dataset.map(tf_serialize_example)
serialized_features_dataset

In [ ]:
def generator():
  for features in features_dataset:
    yield serialize_example(*features)

In [ ]:
serialized_features_dataset = tf.data.Dataset.from_generator(
    generator, output_types=tf.string, output_shapes=())

In [ ]:
serialized_features_dataset

Dan tulis ke dalam TFRecord file:

In [ ]:
filename = 'test.tfrecord'
writer = tf.data.experimental.TFRecordWriter(filename)
writer.write(serialized_features_dataset)

### Membaca file TFRecord

Anda juga dapat membaca file TFRecord menggunakan kelas `tf.data.TFRecordDataset`.

Informasi lebih lanjut tentang penggunaan file TFRecord menggunakan `tf.data` dapat ditemukan [disini](https://www.tensorflow.org/guide/datasets#consuming_tfrecord_data).

Menggunakan `TFRecordDataset`  berguna untuk melakukan standarisasi data input dan mengoptimasi performa.

In [ ]:
filenames = [filename]
raw_dataset = tf.data.TFRecordDataset(filenames)
raw_dataset

Pada titik ini dataset berisi pesan `tf.train.Example` yang terserialisasi. Ketika diiterasi akan mengembalikan skalar tensor string.

Gunakan *method* `.take` untuk menampilkan hanya 10 record pertama.

Catatan: proses iterasi `tf.data.Dataset` hanya bekerja ketika *eager execution* diaktifkan.

In [ ]:
for raw_record in raw_dataset.take(10):
  print(repr(raw_record))

Tensor-tensor ini dapat diuraikan menggunakan fungsi dibawah. Perhatikan bahwa `feature_description` disini dibutuhkan karena dataset menggunakan *graph-execution*, dan memerlukan deskripsi ini untuk membangun bentuk dan tipe asli mereka:

In [ ]:
# Membuat deskripsi dari fitur.
feature_description = {
    'feature0': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'feature1': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'feature2': tf.io.FixedLenFeature([], tf.string, default_value=''),
    'feature3': tf.io.FixedLenFeature([], tf.float32, default_value=0.0),
}

def _parse_function(example_proto):
  # Uraikan input proto `tf.Example` menggunkan dictionary di atas.
  return tf.io.parse_single_example(example_proto, feature_description)

Dengan cara lain, gunakan `tf.parse example` untuk menguraikan seluruh batch sekaligus. Gunakan fungsi ini ke setiap item dalam dataset menggunakan *mehod* `tf.data.Dataset.map`:

In [ ]:
parsed_dataset = raw_dataset.map(_parse_function)
parsed_dataset

Gunakan *eager execution* untuk menampilkan pengamatan pada dataset. Ada 10,000 pengamatan pada dataset ini, tetapi Anda hanya akan menampilkan 10 data pertama. Data ditampilkan sebagai sebuah *dictionary* dari fitur. Setiap item adalah `tf.Tensor`, dan elemen `numpy` pada tensor ini menampilkan nilai dari fitur:

In [ ]:
for parsed_record in parsed_dataset.take(10):
  print(repr(parsed_record))

Di sini, fungsi `tf.parse_example` membuka *field* `tf.Example` menjadi tensor standar.

## File TFRecord di Python

Modul `tf.io` juga berisikan fungsi Python murni untuk membaca dan menulis file-file TFRecord.

### Menulis file TFRecord

Selanjutnya, tuliskan 10,000 pengamatan ke file `test.tfrecord`. Setiap pengamatan dikonversi menjadi sebuah pesan `tf.Example`, kemudian ditulis dalam file. Anda kemudian dapat memeriksa apakah file `test.tfrecord` berhasil dibuat:

In [ ]:
# Menulis pengamatan `tf.Example` kedalam file
with tf.io.TFRecordWriter(filename) as writer:
  for i in range(n_observations):
    example = serialize_example(feature0[i], feature1[i], feature2[i], feature3[i])
    writer.write(example)

In [ ]:
!du -sh {filename}

### Membaca file TFRecord

Tensor yang terserialisasi ini dapat diurai menggunakan `tf.train.Example.ParseFromString`:

In [ ]:
filenames = [filename]
raw_dataset = tf.data.TFRecordDataset(filenames)
raw_dataset

In [ ]:
for raw_record in raw_dataset.take(1):
  example = tf.train.Example()
  example.ParseFromString(raw_record.numpy())
  print(example)

## Panduan: Membaca dan menulis data gambar

Ini adalah contoh proses dari ujung ke ujung bagaimana cara membaca dan menulis data gambar menggunakan TFRecord. Menggunakan gambar sebagai input data, Anda akan menulis data sebagai file TFRecord, kemudian membaca file dan menampilkan gambar tersebut.

Hal ini dapat berguna jika, misalnya, Anda ingin menggunakan berbagai model dengan input dataset yang sama. Dibandingkan menyimpan data gambar secara mentah, data dapat diproses terlebih dahulu menjadi format TFRecord, dan dapat digunakan pada proses lebih lanjut dan pemodelan.

Pertama, mari kita unduh [gambar](https://commons.wikimedia.org/wiki/File:Felis_catus-cat_on_snow.jpg) kucing di salju dan [foto](https://upload.wikimedia.org/wikipedia/commons/f/fe/New_East_River_Bridge_from_Brooklyn_det.4a09796u.jpg) dari Jembatan Williamsburg, NYC yang sedang dalam proses pembangunan.

### Mengambil gambar

In [ ]:
cat_in_snow  = tf.keras.utils.get_file('320px-Felis_catus-cat_on_snow.jpg', 'https://storage.googleapis.com/download.tensorflow.org/example_images/320px-Felis_catus-cat_on_snow.jpg')
williamsburg_bridge = tf.keras.utils.get_file('194px-New_East_River_Bridge_from_Brooklyn_det.4a09796u.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/194px-New_East_River_Bridge_from_Brooklyn_det.4a09796u.jpg')

In [ ]:
display.display(display.Image(filename=cat_in_snow))
display.display(display.HTML('Image cc-by: <a "href=https://commons.wikimedia.org/wiki/File:Felis_catus-cat_on_snow.jpg">Von.grzanka</a>'))

In [ ]:
display.display(display.Image(filename=williamsburg_bridge))
display.display(display.HTML('<a "href=https://commons.wikimedia.org/wiki/File:New_East_River_Bridge_from_Brooklyn_det.4a09796u.jpg">From Wikimedia</a>'))

### Menulis file TFRecord

Sama seperti sebelumnya, lakukan enkode fitur sebagai tipe yang kompatibel dengan `tf.Example`. Hal ini akan menyimpan fitur string dari gambar raw, juga tinggi, lebar, kedalaman , dan fitur `label`. Yang terakhir digunakan ketika Anda menulis file untuk membedakan antara gambar kucing dan gambar jembatan. Gunakan `0` untuk gambar kucing, and `1` untuk gambar jembatan:

In [ ]:
image_labels = {
    cat_in_snow : 0,
    williamsburg_bridge : 1,
}

In [ ]:
# Ini sebuah contoh, hanya menggunakan gambar kucing.
image_string = open(cat_in_snow, 'rb').read()

label = image_labels[cat_in_snow]

# Membuat dictionary dengan fitur yang mungkin relevan.
def image_example(image_string, label):
  image_shape = tf.image.decode_jpeg(image_string).shape

  feature = {
      'height': _int64_feature(image_shape[0]),
      'width': _int64_feature(image_shape[1]),
      'depth': _int64_feature(image_shape[2]),
      'label': _int64_feature(label),
      'image_raw': _bytes_feature(image_string),
  }

  return tf.train.Example(features=tf.train.Features(feature=feature))

for line in str(image_example(image_string, label)).split('\n')[:15]:
  print(line)
print('...')

Perhatikan semua fitur sekarang disimapan dalam pesan `tf.Example`. Selanjutnya, gunakan kode diatas dan menulis contoh pesan ke file yang diberi nama `images.tfrecords`:

In [ ]:
# Tulis file gambar raw ke `images.tfrecords`.
# Pertama, proses dua gambar ke dalam pesan `tf.Example`.
# Kemudian, tulis ke file `.tfrecords`
record_file = 'images.tfrecords'
with tf.io.TFRecordWriter(record_file) as writer:
  for filename, label in image_labels.items():
    image_string = open(filename, 'rb').read()
    tf_example = image_example(image_string, label)
    writer.write(tf_example.SerializeToString())

In [ ]:
!du -sh {record_file}

### Membaca file TFRecord

Anda sekarang mempunyai file `images.tfrecords` dan dapat melakukan iterasi terhadap record yang ada di dalamnya untuk membaca kembali apa yang telah Anda tulis. Mengingat dalam contoh ini anda hanya akan menghasilkan kembali gambar, satu-satunya fitur yang anda butuhkan adalah string dari gambar raw. Ekstrak string menggunakan getters yang telah dideskripsikan di atas, yaitu `example.features.feature['image_raw'].bytes_list.value[0]`. Anda juga dapat menggunakan label untuk menentukan record mana yang kucing dan record mana yang jembatan:

In [ ]:
raw_image_dataset = tf.data.TFRecordDataset('images.tfrecords')

# Membuat dictionary mendeskripsikan fitur
image_feature_description = {
    'height': tf.io.FixedLenFeature([], tf.int64),
    'width': tf.io.FixedLenFeature([], tf.int64),
    'depth': tf.io.FixedLenFeature([], tf.int64),
    'label': tf.io.FixedLenFeature([], tf.int64),
    'image_raw': tf.io.FixedLenFeature([], tf.string),
}

def _parse_image_function(example_proto):
  # Uraikan input proto tf.Example menggunakan dictionary diatas.
  return tf.io.parse_single_example(example_proto, image_feature_description)

parsed_image_dataset = raw_image_dataset.map(_parse_image_function)
parsed_image_dataset

Memulihkan gambar-gambar dari file TFRecord:

In [ ]:
for image_features in parsed_image_dataset:
  image_raw = image_features['image_raw'].numpy()
  display.display(display.Image(data=image_raw))